In [5]:
from dotenv import load_dotenv
import os


from langchain.llms import OpenAI

In [9]:
# Loading the API Key 
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
hf_key = os.getenv("HUGGINGFACE_ACCESS_TOKEN")

Temperature
- Directly influences how creative we want the model to be. This implies that the output probability distribution of the all words are altered to give different/more unique results.
- A value of 0 implies the model is sticking to its safety and only predicting the next most likely word based on max probability 
- A value closer to 1 implies the model is getting more creative. However this could also lead to incorrect outputs or gibberish.


Default Model: GPT 3.5 Turbo

In [10]:
llm = OpenAI(temperature=0.6, api_key=openai_key)

In [11]:
text = 'What is the capital of India'
print(llm.predict(text))



The capital of India is New Delhi.


In [34]:
from langchain import HuggingFaceHub

model_kwargs = {
    'temperature':0,
    'max_length': 64
}

question = 'Can you tell me the capital of Russia?'
flan_model = HuggingFaceHub(repo_id = 'google/flan-t5-large', model_kwargs = model_kwargs, huggingfacehub_api_token=hf_key)
flan_output = flan_model.predict(question)
print(flan_output)

moscow


# Prompt Template

In [35]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
 
capital_pt = PromptTemplate(input_variables=['country'],
               template="Tell me the captial of {country}")

print(capital_pt.format(country = 'India'))

# Create a chain to first get the value for country, add it to the input text using the prompt template and then pass that to the model 
chain = LLMChain(llm = flan_model, prompt = capital_pt)
print(chain.run('India')) 


Tell me the captial of India
arunachal pradesh


# Combining Multiple chains using Sequential Chains

In [38]:
from langchain.chains import SimpleSequentialChain

capital_chain = LLMChain(llm = flan_model, prompt = capital_pt)

famous_template = PromptTemplate(input_variables=['capital'], 
                                 template = "Suggest some great places to visit in {capital}")

famous_chain = LLMChain(llm = flan_model, prompt = famous_template)


final_chain = SimpleSequentialChain(chains = [capital_chain, famous_chain])
final_chain.run('United Kingdom')

'London Eye'

# Sequential Chain  

In [39]:
from langchain.chains import SequentialChain

chosen_model = llm

# Setting up the Template 
capital_pt = PromptTemplate(input_variables=['country'],
               template="Tell me the captial of {country}")
famous_template = PromptTemplate(input_variables=['capital'], 
                                 template = "Suggest some great places to visit in {capital}")

# Setting up the chain
capital_chain = LLMChain(llm = chosen_model, prompt = capital_pt, output_key='capital')
famous_chain = LLMChain(llm = chosen_model, prompt = famous_template, output_key='places')
chain = SequentialChain(chains = [capital_chain, famous_chain], input_variables = ['country'], output_variables = ['capital', 'places'])

# Running the model 
input_text = {'country': 'India'}
chain(input_text)

/home/anshu/miniconda3/envs/llm/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'country': 'India',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': " Some great places to visit in New Delhi are:\n\n1. Red Fort: This iconic red sandstone fort was built in the 17th century and is a must-visit for its beautiful architecture and historical significance.\n\n2. Qutub Minar: Another popular tourist attraction, the Qutub Minar is a 73-meter tall minaret built in the 12th century. It is a UNESCO World Heritage Site and a great place to learn about Delhi's history.\n\n3. India Gate: This war memorial is a popular spot for locals and tourists alike. It is a great place to take a stroll, have a picnic, and admire the beautiful architecture.\n\n4. Lotus Temple: This stunning temple, shaped like a lotus flower, is a symbol of peace and unity. It is open to people of all religions and is a great place to meditate and find inner peace.\n\n5. Humayun's Tomb: This magnificent tomb is another UNESCO World Heritage Site and is known for its stunning Mughal architect

# Chatmodels with ChatOpenAI

In [42]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chatbot = ChatOpenAI(api_key=openai_key, 
                     temperature = 0.6,
                     model = 'gpt-3.5-turbo')

/home/anshu/miniconda3/envs/llm/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [43]:
output = chatbot([
    SystemMessage(content = "You are a travel guide assisstant!"),
    HumanMessage(content = "What are the best cities to visit in Morocco?"),
])
print(output)

/home/anshu/miniconda3/envs/llm/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


content="Morocco is a country with a rich history, vibrant culture, and diverse landscapes. Some of the best cities to visit in Morocco include:\n\n1. Marrakech: Known for its bustling souks, historic medina, and beautiful architecture, Marrakech is a must-visit city in Morocco. Be sure to visit the famous Jemaa el-Fnaa square, the Bahia Palace, and the Majorelle Garden.\n\n2. Fes: Fes is one of the oldest and most well-preserved medieval cities in the world. Explore the maze-like streets of the medina, visit the Bou Inania Madrasa, and see the iconic blue pottery of Fes.\n\n3. Chefchaouen: This picturesque blue city is nestled in the Rif Mountains and is known for its charming streets, blue-washed buildings, and stunning mountain views. It's a great place to relax and explore at a slower pace.\n\n4. Essaouira: This coastal city is a laid-back alternative to the hustle and bustle of Marrakech and Fes. Enjoy fresh seafood, wander through the medina, and relax on the beautiful beaches.\n

# Prompt Template + LLM + Output Parser

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser


In [49]:
class CSV(BaseOutputParser):
    def parse(self, output):
        return output.strip().split(',')
    
template = "You are a helpful assisstant. When the user asks for the best cities to visit in a country, only return top 3 cities in that country separated by commas. No other text is needed"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ('system', template),
    ('human', human_template)
])

chain = chatprompt|chatbot|CSV()
new_output = chain.invoke({'text':'What are the best cities to visit in India?'})

In [50]:
new_output

['Mumbai', ' Delhi', ' Jaipur']